# SpaceNet7 Change Detection
Project task: train neural network for semantic segmentation of urban development change

In [ ]:
! pip install --upgrade rasterio
! pip install -U git+https://github.com/qubvel/segmentation_models.pytorch


# Import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # image plotting
import matplotlib
matplotlib.rcParams['figure.dpi'] = 300 #increase plot resolution

#Raster data handling
from PIL import Image
import skimage
from skimage import io, transform 
import rasterio as rio # geo-oriented plotting library
from rasterio import features
import cv2

#PyTorch
import torch
from torch.utils.data import Dataset, DataLoader, Sampler # custom dataset handling
import torch.autograd.profiler as profiler # to track model inference and detect leaks
from torchvision import transforms, utils
from torch.autograd import Variable
from torch import nn
from torch.nn.modules.padding import ReplicationPad2d
import torchvision.models as models
from torch import optim
from collections import OrderedDict
import segmentation_models_pytorch as smp #semantic segmentation models and utils
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

#Augmentations
import albumentations as alb
from albumentations.pytorch import ToTensorV2

#Logging errors and progress, sending them to tg-bot
import requests
import traceback

#Other
from pathlib import Path # to have path strings as PosixPath objexts
import pathlib 
from pyproj import CRS
import geopandas as gpd # to make dataframes out of geojson files
import itertools
import re
from tqdm.notebook import tqdm
tqdm.pandas()
import os
import gc
from timeit import default_timer as time
import copy
import json
import logging
import datetime




In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Setup

In [ ]:
# Input
train_dir = Path('../input/spacenet-7-multitemporal-urban-development/SN7_buildings_train')
test_dir = Path('../input/spacenet-7-multitemporal-urban-development/SN7_buildings_test_public')
sample_dir = Path('../input/spacenet-7-multitemporal-urban-development/SN7_buildings_train_sample')

# Output
output_path = Path.cwd()
output_csv_path = output_path/'output_csvs/'
Path(output_csv_path).mkdir(parents=True, exist_ok=True)

# Тестовые данные: растры, geojson и geodataframe с интервалом в 24 месяца
test_raster_path = Path('../input/spacenet-7-multitemporal-urban-development/SN7_buildings_train_sample/sample/L15-0506E-1204N_2027_3374_13/images_masked/global_monthly_2018_01_mosaic_L15-0506E-1204N_2027_3374_13.tif')
test_raster_path_24 = Path('../input/spacenet-7-multitemporal-urban-development/SN7_buildings_train_sample/sample/L15-0506E-1204N_2027_3374_13/images_masked/global_monthly_2019_12_mosaic_L15-0506E-1204N_2027_3374_13.tif')
test_geojson_path = Path('../input/spacenet-7-multitemporal-urban-development/SN7_buildings_train_sample/sample/L15-0506E-1204N_2027_3374_13/labels_match_pix/global_monthly_2018_01_mosaic_L15-0506E-1204N_2027_3374_13_Buildings.geojson')
test_geojson_path_24 = Path('../input/spacenet-7-multitemporal-urban-development/SN7_buildings_train_sample/sample/L15-0506E-1204N_2027_3374_13/labels_match_pix/global_monthly_2019_12_mosaic_L15-0506E-1204N_2027_3374_13_Buildings.geojson')
test_gdf = gpd.read_file(test_geojson_path)
test_gdf_24 = gpd.read_file(test_geojson_path_24)

# Делаем из Id индексы датафрейма и сортируем датафрейм по индексу
test_gdf.set_index('Id',inplace=True)
test_gdf_24.set_index('Id',inplace=True)

test_gdf.sort_index(inplace=True)
test_gdf_24.sort_index(inplace=True)



# Metadata Extraction

## Paths metadata
First of all lets collect all existing paths and parse their elements to extract any metadata and significant features.

Typical path looks like this:
../input/spacenet-7-multitemporal-urban-development/SN7_buildings_train/train/L15-1200E-0847N_4802_4803_13/UDM_masks/global_monthly_2018_05_mosaic_L15-1200E-0847N_4802_4803_13_UDM.tif

and can be divided into following parts:
* parent directory: ../input/spacenet-7-multitemporal-urban-development/SN7_buildings_train/train/, which can be reduced as just train, test or sample
* location directory with unique id: L15-1200E-0847N_4802_4803_13 which always has 28 digits
* directory with name that indicates file content: UDM_masks, images, images_masked, labels, labels_match, labels_match_pix
* filename which includes year, month, location id and file content 
* file extension (.tif, .csv and so on)

In [ ]:
def unparse_path(string):
    pattern = r'/(train|test_public|sample)/(L.+)/(\w+)/(.+_(\d+)_(\d+)_m.+_\d+_\d+_\d+)(?:_(\w+))?.(\w+)'
    match = re.findall(pattern=pattern,string=string)
    return match[0]

In [ ]:
sample_string = '../input/spacenet-7-multitemporal-urban-development/SN7_buildings_train/train/L15-0331E-1257N_1327_3160_13/labels_match_pix/global_monthly_2018_01_mosaic_L15-0331E-1257N_1327_3160_13_Buildings.geojson'
unparse_path(sample_string)

Соответственно при заполнении словаря и переносе его в датафрейм будем использовать следующие ключи:
parent, location_dir, sub_dir, filename, year, month, file_type, extension

In [ ]:
def get_paths(path):
    return [str(pth) for pth in Path.glob(path,pattern = '**/*.*')]

pathlist_train = get_paths(train_dir)
pathlist_test = get_paths(test_dir)
pathlist_sample = get_paths(sample_dir)


In [ ]:
print(f'train_files: {len(pathlist_train)} \n test_files: {len(pathlist_test)} \n sample_files: {len(pathlist_sample)}')

In [ ]:
def get_meta_from_pathlist(pathlist):
    keys = ['parent', 'location_dir', 'sub_dir', 'filename', 'year', 'month', 'file_type', 'extension', 'source']
    temp_dir = {key: [] for key in keys}
    for path in pathlist:
        temp_dir['source'].append(path)
        meta = unparse_path(path)
        for i,data in enumerate(meta):
            temp_dir[keys[i]].append(data)
    return pd.DataFrame(temp_dir)

In [ ]:
train_meta_df = get_meta_from_pathlist(pathlist_train)
test_meta_df = get_meta_from_pathlist(pathlist_test)
sample_meta_df = get_meta_from_pathlist(pathlist_sample)
concatenated_df = pd.concat([train_meta_df, test_meta_df, sample_meta_df]).reset_index()

In [ ]:
train_meta_df.head(5)

In [ ]:
test_meta_df.head(5)

In [ ]:
sample_meta_df

In [ ]:
concatenated_df.file_type.value_counts()

In [ ]:
concatenated_df[concatenated_df.file_type == ''].extension.unique()

There are some missing values in file_type. With the elimination method and common sense, we understand that these are rasters.

In [ ]:
train_meta_df.loc[train_meta_df['file_type'] == '', 'file_type'] = 'sat_picture'
test_meta_df.loc[test_meta_df['file_type'] == '', 'file_type'] = 'sat_picture'
sample_meta_df.loc[sample_meta_df['file_type'] == '', 'file_type'] = 'sat_picture'
concatenated_df.loc[concatenated_df['file_type'] == '', 'file_type'] = 'sat_picture'

In [ ]:
concatenated_df.file_type.value_counts()

Now we need to figure out connections between the rasters and masks, udm absence or presence.

In [ ]:
def get_metadata(path):
    pathlist = get_paths(path)
    meta_df = get_meta_from_pathlist(pathlist)
    meta_df.loc[meta_df['file_type'] == '', 'file_type'] = 'Sat_picture'
    
    # Choosing udm masks only and taking the indices
    condition = (meta_df['sub_dir'] == 'UDM_masks')
    udm_indices = meta_df.loc[condition].index
    
    # Get list of unique file names that have UDMs and taking their list
    udm_fnames = list(meta_df.loc[udm_indices,'filename'])
    udm_mask = meta_df['filename'].progress_map(lambda x: x in udm_fnames)
    
    # Initialize has_udm feature as False
    meta_df['has_udm'] = False
    # Changing has_udm feature for files mentioned in list above
    meta_df.loc[udm_mask,'has_udm'] = True

    return meta_df

Generating dataframes with paths metadata

In [ ]:
train_meta_df = get_metadata(train_dir)
test_meta_df = get_metadata(test_dir)
sample_meta_df = get_metadata(sample_dir)
concatenated_meta_df = pd.concat([train_meta_df, test_meta_df, sample_meta_df]).reset_index()

In [ ]:
concatenated_meta_df.has_udm.value_counts()

In [ ]:
train_meta_df.to_csv(output_csv_path/'meta_train.csv')
test_meta_df.to_csv(output_csv_path/'meta_test.csv')
sample_meta_df.to_csv(output_csv_path/'meta_sample.csv')
concatenated_meta_df.to_csv(output_csv_path/'meta_concat.csv')

Now we need to combine untidy dataframe with all existing filepaths to the rasters and masks for different areas of interest. It will help us to form custom torch dataset for model training and validation. Lets recollect how the data is organized:

* there is only images_masked dir inside the test_public
* train and sample consist of: images, images_masked, labels, labels_match, labels_match_pix dirs
* next part of the path is the same for all dataset parts
* despite the paths as adresses we need to mention separately: parent directory as dataset part(train, test, sample), AOI ids, date of the shot, filename 
* there are two kinds of labels: with _Buildings and _UDM postfixes. _Buildings postfix is clear. UDM - unidentified mask and most probably contain glitches related to clouds, aerosols and so on

In [ ]:
concatenated_meta_df[concatenated_meta_df.has_udm == True].filename.iloc[0]

In [ ]:
def make_untidy(ser):
    """Function for making untidy data out of tidy dataframe"""
    part = ser['parent']
    aoi = ser['location_dir']
    year = ser['year']
    month = ser['month']
    filename = ser['filename']
    has_udm = ser['has_udm']
    
    #making images_masked adresses
    images_masked =  aoi + '/images_masked/' + filename + '.tif'
    
    # test public has images_masked only
    if part == 'test_public':
        images = None
        labels = None
        labels_match = None
        labels_match_pix = None
        UDM_masks = None
    else:
        images = aoi + '/images/' + filename + '.tif'
        labels = aoi + '/labels/' + filename + '.geojson'
        labels_match = None
        labels_match_pix = None
        
        if has_udm == True:
            UDM_masks = aoi + '/UDM_masks/' + filename + '_UDM.tif'
        else: UDM_masks = None
        
        images = aoi + '/images/' + filename + '.tif'
        labels_buildings = aoi + '/labels/' + filename + '_Buildings.geojson'
        labels_udm = aoi + '/labels/' + filename + '_UDM.geojson'
        labels_match = aoi + '/labels_match/' + filename + '_Buildings.geojson'
        labels_match_pix = aoi + '/labels_match_pix/' + filename + '_Buildings.geojson'
        
    keys = ['part', 'aoi', 'filename', 'year', 'month', 'has_udm', 'UDM_masks', 'images', 'images_masked', 'labels', 'labels_match', 'labels_match_pix'] 
    values = [part, aoi, filename, year, month, has_udm, UDM_masks, images, images_masked, labels, labels_match, labels_match_pix]
    temp_dict = {k:v for k, v in zip(keys,values)}
    return pd.Series(temp_dict)

def make_untidy_dataframe(df):
    return df.progress_apply(lambda x: make_untidy(x), axis = 1).drop_duplicates()

In [ ]:
train_untidy_df = make_untidy_dataframe(train_meta_df)
test_untidy_df = make_untidy_dataframe(test_meta_df)
sample_untidy_df = make_untidy_dataframe(sample_meta_df)
concatenated_untidy_df = pd.concat([train_untidy_df, test_untidy_df, sample_untidy_df]).reset_index()

In [ ]:
train_untidy_df.to_csv(output_csv_path/'untidy_train.csv')
test_untidy_df.to_csv(output_csv_path/'untidy_test.csv')
sample_untidy_df.to_csv(output_csv_path/'untidy_sample.csv')
concatenated_untidy_df.to_csv(output_csv_path/'untidy_concat.csv')

# Extracting meta-data from file names
As we mentioned above, filenames have similar structure

In [ ]:
label_csv_path = Path('../input/spacenet-7-multitemporal-urban-development/SN7_buildings_train_csvs/csvs/sn7_train_ground_truth_pix.csv')
df = pd.read_csv(label_csv_path)

In [ ]:
df.sample(5)

In [ ]:
print(df.filename.unique())
print(len('L15-0506E-1204N_2027_3374_13'))

global_monthly_YYYY_MM_mosaic_AOI_ID.tif

где:
* YYYY - 4 digits number of the year
* MM - 2 digits number of the month
* AOI_ID - area of interest, location id (28 digits), upper register letters.

Lets make helpers to be able to extract these features from filenames as well.

In [ ]:
def extract_fname_feats(string):
    """Function that extract metadata from filenames by regular expressions"""
    date_pattern = r'\d+'
    id_pattern = r'_(L.+)'
    extract = lambda x, y: re.findall(pattern=x,string=y)
    date = tuple(extract(date_pattern, string)[0:2])
    aoi = extract(id_pattern, string)[0]
    
    feat_dict = {'date':date,
                'aoi': aoi.replace('.tif','')}
    return feat_dict
    

In [ ]:
test_str = 'global_monthly_2019_12_mosaic_L15-0506E-1204N_2027_3374_13.tif'
feats = extract_fname_feats(test_str)

In [ ]:
df['date'], df['aoi'] = zip(*df['filename'].progress_apply(lambda x: list(extract_fname_feats(x).values())))
df['year'], df['month'] = zip(*df['date'].progress_map(lambda x: x[0:2]))
df['date'] = df['date'].progress_apply(lambda x: datetime.date(int(x[0]), int(x[1]), 1))

Now we can to get a closer look to the dataset.

In [ ]:
df.aoi.value_counts()


In [ ]:
df.year.value_counts()


In [ ]:
print(f'Date range: {df.date.min()} - {df.date.max()}')

## Raster metadata

In [ ]:
test_gdf.head(5)

Geodataframe includes polygons represented in WKT format (well-known-text), these are primitives to draw bulding masks. df columns are district, raster path, polygon geometry.

Lets inspect the rasters' size, number of channels, georeferncing, projection, affine transformations

In [ ]:
raster = rio.open(test_raster_path)
print(raster.meta)
print(raster.crs.wkt)
r = raster.read()
print(r.shape)

* Raster size: 1024 x 1024 px, 3-4 channels: RGB/alpha-RGB
* Projection: WGS 84 / Pseudo-Mercator (EPSG:3857)
* Affine: affine transformation matrix, 6 numbers: a,b,c,d,e,f which make this matrix

| x' |   | a b c | | x |  
| y' | = | d e f | | y |  
| 1  |   | 0 0 1 | | 1 |  

 x,y - pixel coordinates, x',y' - world coordinates

# Helpers

Roughly speaking, we need to be able to:
* plot poligons (separatly and above rasters)
* make rasters out of vector polygons
* subtract masks to get change masks
* make smaller chips out of original pictures

In [ ]:
def plot_polygons(gdf, fill = False, l_wd = 0.2, axs = None, color = 'yellow'):
    """Function to plot polions of structures
    gdf: geodataframe with polygons geometry
    fill: boolean, defines either polygons filled or not
    l_wd: float, linewidth of the polygon contour
    axs: ax, variable to specify already existing ax to plot, for example: to overlay a sattelite picture"""
    
    if axs == None:
        _, ax = plt.subplots(1, figsize = (3,3))
    
    for geom in gdf.geometry:
        if fill:
            # for highliting above the raster
            ax = axs
            patch = PolygonPatch(geom,color=color, linewidth = l_wd)
            ax.add_patch(patch)
        else:
            ax.plot(*geom.exterior.xy,linewidth=l_wd)
    return(ax)

In [ ]:
plot_polygons(test_gdf)

In [ ]:
def plot_satellite(path, gdf = None, fill = False, l_wd = 0.2):
    """Function to plot sattelite image with ability to overlay gdf polygons
    path: string, path to raster
    gdf: geodataframe that we are going to use to plot polygons
    fill: boolean, either polygons are filled or not
    l_wd: float, linewidth"""
    
    fig, ax = plt.subplots(1, figsize = (3,3))
    fig.tight_layout() #to adjust subplots layout
    
    sat = rio.open(path)
    sat = sat.read()
    sat = sat.transpose((1,2,0,))
    ax.imshow(sat)
    
    if gdf is not None:
        plot_polygons(gdf,fill=fill,axs=ax,l_wd=l_wd)
        
    return(ax)

    

In [ ]:
plot_satellite(test_raster_path)

In [ ]:
plot_satellite(test_raster_path, test_gdf, fill = True)

In [ ]:
def make_mask(source, raster):
    return features.rasterize(((polygon, 255) for polygon in source['geometry']),out_shape=raster.shape)
    

def rasterize_mask(source, raster_path, ftype = 'gdf'):
    
    """Function that allows us to make rasterized mask
    ftype: string, 'gdf' or 'geojson', defines what is used for source of geometry: dataframe or json
    source: string or pandas dataframe object, path to geojson or dataframe vatiable
    raster_path: path to raster which is used as reference to mask shape
    """
    
    if ftype == 'gdf':
        with rio.open(raster_path) as raster:
            r = raster.read(1)
            mask = make_mask(source, r)
        return mask
            
    elif ftype == 'geojson':
        gdf = gpd.read_file(source)
        with rio.open(raster_path) as raster:
            r = raster.read(1)
            mask = make_mask(gdf, r)
        return mask
    
    else:
        raise ValueError('ftype is incorrect, it can be either "gdf" or "json"')
        

In [ ]:
test_mask = rasterize_mask(test_geojson_path, test_raster_path, ftype = 'geojson')
test_mask_24 = rasterize_mask(test_geojson_path_24, test_raster_path_24, ftype = 'geojson')
masks = [test_mask, test_mask_24]
months = ['month_1', 'month_24']

In [ ]:
print(f'Mask data type {type(test_mask)}, unique values in mask - {np.unique(test_mask)}, mask shape - {test_mask.shape}')

0 - background, 255 - mask(building) needs to be normalised (divided by 255)
Lets see the very first and the very last sample masks (2 years interval)

In [ ]:
_, axs = plt.subplots(1,2, figsize = (10,10))
_.tight_layout()
for i, ax in enumerate(axs):
    ax.set_title(months[i])
    ax.imshow(masks[i])

We see some difference with naked eye already. Lets make helper and try to see if there is any need to track which of the buildings are demolished through time.

In [ ]:
def gdf_difference(gdf1, gdf2):
    """Function that leaves only structures which make difference
    between two versions of the geodataframe 
    
    Args:
    gdf1: first geodataframe with polygons (earlier one)
    gdf2: second geodataframe with polygons (later one)
    Return:
    gdf with polygons that make difference
    """
    try:
        gdf1.reset_index(inplace=True,drop=True)
    except:
        pass
    try:
        gdf2.reset_index(inplace=True,drop=True)
    except:
        pass
    
    
    len_1 = len(gdf1)
    len_2 = len(gdf2)
    
    len_diff = abs(len_2-len_1)
    
    if len_2 > len_1:
        start_index = len_2-len_diff
        diff_gdf = gdf2[start_index:].copy()
    else:
        start_index = len_1-len_diff
        diff_gdf = gdf1[start_index:].copy()

    diff_gdf.reset_index(inplace=True,drop=True)
        
    return diff_gdf

In [ ]:
def get_difference(gdf1, gdf2):
    
    """Function that leaves only new structures between two versions of the geodataframe 
    and classifies either structure is built or demolished
    
    Args:
    gdf1: first geodataframe with polygons (earlier one)
    gdf2: second geodataframe with polygons (later one)
    Return:
    gdf with polygons that make difference
    """
    
    df1 = gdf1.copy()
    df2 = gdf2.copy()
    df1.drop(['area','image_fname','iou_score'], inplace = True, axis = 1, errors = 'ignore')
    df2.drop(['area','image_fname','iou_score'], inplace = True, axis = 1, errors = 'ignore')
    df1['time'] = 'before'
    df2['time'] = 'after'
    
    difference = df1.merge(df2, how = 'outer', on = 'geometry')
    difference = difference[(difference.time_x.isna() == True) | (difference.time_y.isna() == True)]
    difference['mark'] = difference['time_x'].apply(lambda x: 'before' if x == 'before' else 'after')
    difference.drop(['time_x', 'time_y'], axis = 1, inplace = True)
    return difference

In [ ]:
diff = get_difference(test_gdf, test_gdf_24)

Now we now which buildings are appeared and which are demolished. There are much fewer demolished buildings, and for the sake of simplicity I will neglect them.

In [ ]:
builded = diff[diff['mark'] == 'after']
destroyed = diff[diff['mark'] == 'before']

In [ ]:
bld_diff = rasterize_mask(builded,test_raster_path)
dst_diff = rasterize_mask(destroyed,test_raster_path)

In [ ]:
_,axs = plt.subplots(1,4,figsize=(10,10))
_.tight_layout()

masks = [test_mask,test_mask_24, bld_diff, dst_diff]
titles = ['month 1', 'month 24', 'builded', 'destroyed']

for i,ax in enumerate(axs):
    ax.set_title(titles[i])
    ax.imshow(masks[i]);

Now we need to make chipmaker to inflate dataset and be able to train deeper model.

In [ ]:
class ChipCreator():
    """Class that allows us to split bigger satellite picture and mask into smaller pieces"""
    
    def __init__(self, dimension, is_raster = False):
        self.dimension = dimension
        self.raster = is_raster
        
    def __read_image(self,image):
        # checks whether image is a path or array
        if isinstance(image,(pathlib.PurePath,str)):
                with Image.open(image) as img:
                    # converts image into np array
                    np_array = np.array(img)
                return np_array
            
        elif isinstance(image,np.ndarray):
            return image
        else:
            raise ValueError(f"Expected Path or Numpy array received: {type(image)}")
        
    def make_chips(self, image):
        
        #getting image and converting to np.array if necessary
        np_array = self.__read_image(image)
        
        # then get numbers of chips per row and column
        n_rows = (np_array.shape[0] - 1) // self.dimension + 1
        n_cols = (np_array.shape[1] - 1) // self.dimension + 1
        
        chip_list = [] #
        for r in range(n_rows):
            for c in range(n_cols):
                #starting row and column
                start_r_idx = r*self.dimension
                end_r_idx = start_r_idx + self.dimension
                #ending row and column
                start_c_idx = c*self.dimension
                end_c_idx = start_c_idx + self.dimension
                #cutting fragment by indexes
                chip = np_array[start_r_idx:end_r_idx,start_c_idx:end_c_idx]
                
                if self.raster:
                    # if raster is True then format is (channels, rows, columns)
                    # else (rows, columns, channels)
                    chip = np.moveaxis(chip,-1,0)

                chip_list.append(chip)

        return np.array(chip_list)
    def __call__(self, image):
        # slightly different verison of make_chips
        np_array = self.__read_image(image)
        n_rows = (np_array.shape[1] - 1) // self.dimension + 1
        n_cols = (np_array.shape[2] - 1) // self.dimension + 1
        chip_dict = {'chip':[],'x':[],'y':[], 'blank':[]}
        for r in range(n_rows):
            for c in range(n_cols):
                start_r_idx = r*self.dimension
                end_r_idx = start_r_idx + self.dimension

                start_c_idx = c*self.dimension
                end_c_idx = start_c_idx + self.dimension
                chip = np_array[:,start_r_idx:end_r_idx,start_c_idx:end_c_idx]

                chip_dict['chip'].append(chip)
                chip_dict['x'].append(start_r_idx)
                chip_dict['y'].append(start_c_idx)
                if chip.mean() == 0 and chip.sum() == 0:
                    chip_dict['blank'].append('_blank')
                else:
                    chip_dict['blank'].append('')
        return chip_dict
    
def plot_many(pictures, ncols = 4, dpi = 300, is_raster = False):
    matplotlib.rcParams['figure.dpi'] = dpi
    nrows = (len(pictures) - 1) // ncols + 1
    
    fig,axs = plt.subplots(nrows,ncols,figsize=(10,10))
    fig.tight_layout()
    
     
    for r,ax in enumerate(axs):
        for c,row in enumerate(ax):
            # i is current index in array of axes
            i = r*ncols + c
            ax[c].set_title(i)
            image = pictures[i]
            # unmaking raster format if necessary
            if is_raster:
                image = np.moveaxis(image,0,-1)
            ax[c].imshow(image);

    

Lets check how it works.

In [ ]:
сhips_256 = ChipCreator(256, is_raster = True)
plot_many(сhips_256.make_chips(test_raster_path), is_raster = True)

In [ ]:
plot_many(сhips_256.make_chips(test_mask), is_raster = True)

# Torch Custom Dataset

In [ ]:
root_sample_folder =  Path('../input/spacenet-7-multitemporal-urban-development/SN7_buildings_train_sample/sample')
csv_sample_path = Path('./output_csvs/untidy_sample.csv')
root_train_folder =  Path('../input/spacenet-7-multitemporal-urban-development/SN7_buildings_train/train')
csv_train_path = Path('./output_csvs/untidy_train.csv')

Сведем в класс датасета еще функции написанные выше (генератор чипсов, вычисление разницы в масках)

In [ ]:
class CustomSatelliteDataset():
    """SpaceNet7 dataset imagery holder"""
    
    def __init__(self, csv, root_folder, udm = False, transform = None, chip_dim = None):
        """
        csv: string, path to the csv file with untidy dataframe with all elements adresses
        root_folder: string, path where imagery is stored
        udm_use: boolean, condition which specify using of udm masks
        transform: np.array, transformation matrix if needed
        chip_dim: int, pixel dimension of the chip size"""
        
        self.csv = pd.read_csv(csv)
        self.root_folder = root_folder
        self.udm_use = udm
        self.transform = transform
        self.chip_dim = chip_dim
        
        
        if chip_dim is not None:
            self.сhip_gen = self.__ChipGenerator(dimension = self.chip_dim)
            # в этом датасете все картинки 1024Х1024
            self.n_chips = ((1024 - 1) // self.chip_dim + 1)**2
        
        self.idx_combinations = self.__get_all_idx_combinations()
        self.max = self.__len__()
            
        
    def __len__(self):
        """Function that returns the dataset length including chip division"""
        if self.chip_dim is not None:
            return len(self.idx_combinations)*self.n_chips
        else:
            return len(self.idx_combinations)
    
    def __getitem__(self, idx):
        """Function for read images as required, but not to store them all in memory"""
        #checking chip division
        if self.chip_dim is not None:
            img_idx = idx//self.n_chips
            chip_idx = idx%self.n_chips
        else:
            img_idx = idx
        
        #checking is idx numer or tensor
        if torch.is_tensor(img_idx):
            img_index = img_index.to_list()
        # getting two images    
        idx1,idx2 = self.idx_combinations[img_idx]
        # and their path
        img1_path = self.root_folder/self.csv.loc[idx1,'images_masked']
        img2_path = self.root_folder/self.csv.loc[idx2,'images_masked']
        # getting building polygons
        labels1_path = self.root_folder/self.csv.loc[idx1,'labels_match_pix']
        labels2_path = self.root_folder/self.csv.loc[idx2,'labels_match_pix']
        # reading rasters from the paths
        with rio.open(img1_path) as r1, rio.open(img2_path) as r2:
            raster1 = r1.read()[0:3]  
            raster2 = r2.read()[0:3]
            raster_bounds = r1.bounds
            rio_transform = r1.transform
        # to get difference we pass two concatenated images
        raster_diff = np.concatenate((raster1,raster2),axis=0)
        # getting  their dates 
        date1 = tuple(self.csv.loc[idx1,['month','year']])
        date2 = tuple(self.csv.loc[idx2,['month','year']])
        # read geojson files fwith polygons
        gdf1 = gpd.read_file(labels1_path).set_index('Id').sort_index()
        gdf2 = gpd.read_file(labels2_path).set_index('Id').sort_index()
        
        # этих пока нет (дописал)
        # get the change between the 2 satellite images by comparing their polygons
        gdf_diff = self.__geo_difference(labels1_path,labels2_path)
        # get the corresponding rasterized image of the geodataframes
        mask_diff = self.__rasterize_gdf(gdf_diff,out_shape=raster1.shape[1:3])   
        
        sample = {'raster1':raster1,'raster2':raster2,'raster_diff':raster_diff,'raster_bounds':raster_bounds,'rio_transform':rio_transform,
                  'date1':date1,'date2':date2,'mask_diff':mask_diff,'fname':img1_path.parent.parent.stem}
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
        
        
    
    def  __get_all_idx_combinations(self):
        
        all_combinations = []
        # group by area of interest
        aoi_groups = self.csv.groupby('aoi')
        # loop through the groups and get the different index combinations
        for i,aoi in enumerate(aoi_groups):
            # get the dataframe in the group
            loc_frame = aoi[1]
            # excluding unidentified masks
            condition = (loc_frame['has_udm'] == False)
            # return a list of the indices in the location dataframe
            l = list(loc_frame[condition].index)
            # use itertools to get all the different combinations between 2 in the list
            combinations = list(itertools.combinations(l,2))
            all_combinations.extend(combinations)
        return all_combinations
    
    def __geo_difference(self,geojson1,geojson2):
        # read geojson into geodataframes
        gdf1 = gpd.read_file(geojson1).set_index('Id').sort_index()
        gdf2 = gpd.read_file(geojson2).set_index('Id').sort_index()

        # get geodataframe lengths
        len_1 = len(gdf1)
        len_2 = len(gdf2)
        # check which gdf is longer
        len_diff = abs(len_2-len_1)

        if len_2 > len_1:
            start_index = len_2-len_diff
            diff_gdf = gdf2.iloc[start_index:].copy()
        else:
            start_index = len_1-len_diff
            diff_gdf = gdf1.iloc[start_index:].copy()

        # reset the index
        diff_gdf.reset_index(inplace=True,drop=True)

        return diff_gdf

    
    def __rasterize_gdf(self,gdf,out_shape):
        # if geodataframe is empty return empty mask
        if len(gdf)==0:
            return np.zeros((1,*out_shape))
            
        mask = features.rasterize(((polygon, 255) for polygon in gdf['geometry']),out_shape=out_shape)
        
        return np.expand_dims(mask,axis=0)
    
    class __ChipCreator():
        """Class that allows us to split bigger satellite picture and mask into smaller pieces"""
        # при попытке воспользоваться попробую посмотреть на форму массива, 4 канала в начале или в конце и нужно ли двигать ось    
        def __init__(self, dimension):
            self.dimension = dimension
            self.chip_dict = {'chip':[],'x':[],'y':[], 'blank':[]}
        
        def __read_image(self,image):
            # checks whether image is a path or array
            if isinstance(image,(pathlib.PurePath,str)):
                with Image.open(image) as img:
                    # converts image into np array
                    np_array = np.array(img)
                    return np_array
            
            elif isinstance(image,np.ndarray):
                return image
            else:
                raise ValueError(f"Expected Path or Numpy array received: {type(image)}")
        
        def __call(self, image):
        
            #getting image and converting to np.array if necessary
            np_array = self.__read_image(image)
            chip_dict = {'chip':[],'x':[],'y':[], 'blank':[]}
        
            # then get numbers of chips per row and column
            n_rows = (np_array.shape[0] - 1) // self.dimension + 1
            n_cols = (np_array.shape[1] - 1) // self.dimension + 1
        
            for r in range(n_rows):
                for c in range(n_cols):
                    #starting row and column
                    start_r_idx = r*self.dimension
                    end_r_idx = start_r_idx + self.dimension
                    #ending row and column
                    start_c_idx = c*self.dimension
                    end_c_idx = start_c_idx + self.dimension
                    #cutting fragment by indexes
                    chip = np_array[:, start_r_idx:end_r_idx,start_c_idx:end_c_idx]
                    #filling dictionary
                    chip_dict['chip'].append(chip)
                    chip_dict['x'].append(start_r_idx)
                    chip_dict['y'].append(start_c_idx)
                    
                    # Marking blank chips
                    if chip.mean() == 0 and chip.sum() == 0:
                        chip_dict['blank'].append(1)
                    else:
                        chip_dict['blank'].append(0)

            return chip_dict

In [ ]:
class DatasetCreator():
    def __init__(self,chip_dimension=256):
        self.chip_dimension = chip_dimension
    
    def __call__(self,dataset):
        for d in tqdm(dataset):
            raster_diff = d['raster_diff']
            mask_diff = d['mask_diff']
            
            self.__fname = d['fname']
            self.__date1= d['date1']
            self.__date2 = d['date2']
            self.__raster_bounds = d['raster_bounds']
            self.__transform = d['rio_transform']
            self.__raster_shape = raster_diff.shape[1:3]
            
            
            
            self.__save_chips(image=raster_diff,subdir_name='chips')
            self.__save_chips(image=mask_diff,subdir_name='masks')
            
            
    def __save_chips(self,image,subdir_name='chips'):
        
        month1,year1 = self.__date1
        month2,year2 = self.__date2
        
        chip_generator = ChipCreator(dimension=self.chip_dimension)
        chip_dict = chip_generator(image)
        
        x_coords = chip_dict['x']
        y_coords = chip_dict['y']
        chips = chip_dict['chip']
        blanks = chip_dict['blank'] 
        
        im_dir = Path('chip_dataset/change_detection')/Path(self.__fname)/Path(subdir_name)/Path(f'{year1}_{month1}_{year2}_{month2}')
        im_dir.mkdir(parents=True, exist_ok=True)

        for chip,x,y,blank in zip(chips,x_coords,y_coords,blanks):
            im_name = f'global_monthly_{year1}_{month1}_{year2}_{month2}_chip_x{x}_y{y}_{self.__fname}{blank}.tif'
            im_path = im_dir/im_name
            
            if subdir_name == 'chips':
                count = 6
            else:
                count = 1
            
            # Calculate the new bounds for the raster chips
            transform = self.__get_geo_transform(x,y)
            
            profile = {'driver':'GTiff', 'width':self.chip_dimension,'height':self.chip_dimension,'crs':CRS.from_wkt('LOCAL_CS["WGS 84 / Pseudo-Mercator",UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3857"]]'),
                       'count':count,'dtype':rio.uint8, 'compress':'zip','transform': transform}
            
            with rio.open(im_path, 'w',**profile) as dst:
                dst.write(chip.astype(rio.uint8))
                
    
    def __get_geo_transform(self,x,y):
        top = self.__raster_bounds[3]
        bottom = self.__raster_bounds[1]
        left = self.__raster_bounds[0]
        right = self.__raster_bounds[2]
        
        raster_height = self.__raster_shape[0]
        raster_width = self.__raster_shape[1]
        
        chip_height = (top-bottom)/(raster_height//self.chip_dimension)
        chip_width = (left-right)/(raster_width//self.chip_dimension)
        
        pixel_height = (top-bottom)/raster_height
        pixel_width = (left-right)/raster_width
        
        chip_top = top + y * pixel_height
        chip_bottom = chip_top + chip_height
        
        chip_left = left + x * pixel_width
        chip_right = chip_left + chip_width
        
        bounds = {'left': chip_left, 'bottom': chip_bottom, 'right': chip_right, 'top': chip_top}
        
        return rio.Affine(self.__transform[0], self.__transform[1], chip_left,self.__transform[3], self.__transform[4], chip_top)

In [ ]:
#dataset = CustomSatelliteDataset(root_folder=root_train_folder,csv=csv_train_path)

In [ ]:
#dataset_creator = DatasetCreator(chip_dimension=64)

In [ ]:
#dataset_creator(dataset=dataset)

Dividing the dataset into chips, according to tqdm's calculations, takes 194 hours. Anyway, after I have learned and reproduced the preparation process, for speed purposes I will take the dataset which is already split into chips the same way. Fortunately it exists and is open on the kaggle.

In [ ]:
gc.collect()

# Deep Learning preparation
Now we operate the change detection dataset we prepared above.
It is necessary to:
* redefine paths
* divide dataset into train/test/valid parts and avoid leakage
* augment the data without distortion. rotation, reflection, padding, normalisation, convertation to tensor format.
* make torch dataset with __len__ and __getitem__ attributes, to be able to lazyload and not to store nothing.
* consider unchanged chips and may be avoid them, make balancing sampler.
* choose model architecture, learning rate policy, loss and metrics. considering specificity of the task -  Jaccard loss and Intersection over Union (IoU) is most relevant.
* maintain logging and messaging to tg bot.




In [ ]:
root_folder = '../input/spacenet-7-change-detection-chips-and-masks/chip_dataset/chip_dataset/change_detection/'
csv_path = '../input/spacenet-7-change-detection-chips-and-masks/annotations.csv'

BATCH_SIZE = 64
NUM_WORKERS = 8

In [ ]:
df = pd.read_csv(csv_path)
df.sample(5)

In [ ]:
df.target.mean()

In [ ]:
# проверка что таргет = 0 действительно пустые
df[df.target == 0].is_blank.value_counts()

only 17% contains change, we need at least 50%, but probably less or no blank masks.

## Train/Test/Valid split

In [ ]:
aoi = df['im_name'].unique()
len(aoi)

60 unique locations, mentioned as im_name in this df. Lets divide into train/test parts by whole locations to avoid leakage.
40/10/10

In [ ]:
train_aoi = aoi[:40]
test_aoi = aoi[-20:-10]
valid_aoi = aoi[-10:]

In [ ]:
def choose_aoi(df, names):
    mask = df['im_name'].map(lambda x: x in names)
    return df[mask].reset_index(drop=True)

df_dict = {'train' : choose_aoi(df, train_aoi),
          'test' : choose_aoi(df, test_aoi),
          'valid' : choose_aoi(df, valid_aoi)
          }

del(df)

In [ ]:
class TorchDataset(Dataset):
    """Dataset class
    Args:
        root_folder: Path object, root directory of picture dataset
        csv: pandas.DataFrame, untidy df with all data relationships
        aug: albumentations dictionary
        preproc: callable, preprocessing function related to specific encoder
        grayscale: boolean, preprocessing condition to grayscale colored rasters
    Return:
        image, mask tensors"""
    
    def __init__(self, root_folder, df, aug = None, preproc = None, grayscale = True):
        self.root_folder = root_folder
        self.csv = df
        self.aug = aug
        self.preproc = preproc
        self.grayscale = grayscale
    
    def __len__(self):
        return len(self.csv)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        chip_path = self.root_folder + self.csv.loc[idx,'chip_path']
        # read chip into numpy array
        chip = skimage.io.imread(root_folder + self.csv.loc[idx,'chip_path']).astype('float32')
        if self.grayscale:
            gray1 = np.dot(chip[:,:,0:3], [0.2989, 0.5870, 0.1140])
            gray2 = np.dot(chip[:,:,3:], [0.2989, 0.5870, 0.1140])
            chip = np.divide(np.stack((gray1, gray2),axis = 2),255).astype('float32')
        # get target for corresponding chip
        mask = np.abs(np.divide(skimage.io.imread(root_folder + self.csv.loc[idx,'mask_path']),255)).astype('float32')
        # apply augmentations
        if self.aug:
            sample = self.aug(image=chip, mask=mask)
            image, mask = sample['image'], sample['mask']
            mask = mask.unsqueeze(0)
            if self.grayscale:
                sample = {'I1':image[0,:,:].unsqueeze(0),'I2':image[1,:,:].unsqueeze(0), 'label':mask}
            else: 
                sample = {'I1':image[0:3,:,:],'I2':image[3:,:,:], 'label':mask}
            del(image,mask,chip,gray1,gray2)
            return sample
        else:
            image = torch.Tensor(np.moveaxis(chip, 2, 0))
            mask = torch.Tensor(mask).unsqueeze(0)
            if self.grayscale:
                sample = {'I1':image[0,:,:].unsqueeze(0),'I2':image[1,:,:].unsqueeze(0), 'label':mask}
            else: 
                sample = {'I1':image[0:3,:,:],'I2':image[3:,:,:], 'label':mask}
            del(mask,chip,gray1,gray2)
            return sample
    

    
    


class BalancedSampler(Sampler):
    """Balancer for torch.DataLoader to adjust chips loading"""
    
    def __init__(self, dataset, percentage = 0.5):
        """
        dataset: custom torch dataset
        percentage: float number between 0 and 1, percentage of change containing pictures in batch
        """
        assert 0 <= percentage <= 1,'percentage must be a value between 0 and 1'
        
        self.dataset = dataset
        self.pct = percentage
        self.len_ = len(dataset)
    
    def __len__(self):
        return self.len_
    
    def __iter__(self):
        # get indices for chips containing change and blank ones
        change_chip_idxs = np.where(self.dataset.csv['target'] == 1)[0]
        blank_chip_idxs = np.where(self.dataset.csv['target'] == 0)[0]
        # randomly sample from the incides of each class according to percentage value
        change_chip_idxs = np.random.choice(change_chip_idxs,int(self.len_ * self.pct), replace=True)
        blank_chip_idxs = np.random.choice(blank_chip_idxs,int(self.len_ * (1 - self.pct))+1, replace=False)
        # stack and shuffle of sampled indices
        all_idxs = np.hstack([change_chip_idxs,blank_chip_idxs])
        np.random.shuffle(all_idxs)
        return iter(all_idxs)

## Augmentations

In [ ]:
chip_dimension = 64
augs = {
    'train': alb.Compose([
        alb.PadIfNeeded(min_height=chip_dimension,min_width=chip_dimension,value=0,p=1),
        alb.HorizontalFlip(p=0.5),
        alb.VerticalFlip(p=0.5),
        ToTensorV2() #apparently doesn't work properly with smp Unet, included in get_preprocessing function
    ]),
    'test': alb.Compose([
        alb.PadIfNeeded(min_height=chip_dimension,min_width=chip_dimension,value=0,p=1),
        ToTensorV2()
    ]), 
    'valid': alb.Compose([
        alb.PadIfNeeded(min_height=chip_dimension,min_width=chip_dimension,value=0,p=1),
        ToTensorV2()
    ]),
}

In [ ]:
augs['train']

# Model setup

# Metrics evaluation and logging
## Tg-bot setup

In [ ]:
import ast
file = open("../input/logging-utils/credentials.txt", "r")
contents = file.read()
token = ast.literal_eval(contents)
file.close()

def telegram_bot_sendtext(bot_message):
    send_text = 'https://api.telegram.org/bot' + token['bot_token'] + '/sendMessage?chat_id=' + token['bot_chatID'] + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()

def telegram_send_file (file_address):
    url = f'https://api.telegram.org/bot' + token['bot_token'] + '/sendVoice'
    #response = requests.post(url, data=data)
    post_data = {'chat_id': token['bot_chatID']}
    with open(file_address, 'r+b') as infile:
        post_file = {'document': infile}
        r = requests.post(f'https://api.telegram.org/bot' +token['bot_token'] + '/sendDocument', data=post_data, files=post_file)
        print(r.text)


test = telegram_bot_sendtext("Testing Telegram bot")
print(test)

def log_traceback(ex, ex_traceback=None):
    if ex_traceback is None:
        ex_traceback = ex.__traceback__
    tb_lines = [ line.rstrip('\n') for line in
                 traceback.format_exception(ex.__class__, ex, ex_traceback)]
    return tb_lines

In [ ]:
dic = {'a':'b'}
with open('d.json', 'w+') as f:
    json.dump(dic, f, indent=4) 
telegram_send_file('./d.json')

## Metrics Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix

class IoULoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoULoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions 
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection 
        
        IoU = (intersection + smooth)/(union + smooth)
                
        return 1 - IoU

def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
    """Fast enough iou calculation function"""
    SMOOTH = 1e-6
    outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    #outputs = outputs.detach()
    #labels = labels.detach()
    
    intersection = (outputs & labels).float().sum((1, 2))
    union = (outputs | labels).float().sum((1, 2))
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    
    thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    
    return thresholded.mean() # to get a batch average

def segmentation_report(running_preds, running_labels):
    """Function to get a closer look to a confusion metrics and related metrics"""
    rp = running_preds.flatten()
    rl = running_labels.flatten()
    tn, fp, fn, tp = confusion_matrix(rl, rp, labels=[0,1]).ravel()
    px_accuracy = (tp+tn) / (tp+fp+tn+fn)
    precision = tp / (tp+fp)
    recall = tp / (tp+fn)
    #calculating intersection over union
    intersection = np.logical_and(rl, rp)
    union = np.logical_or(rl, rp)
    iou_score = np.sum(intersection) / np.sum(union)
    fmeasure = 2 * precision * recall / (precision + recall)
    #making report
    report = { 'tp/tn/fp/fn' : (tp,tn,fp,fn),
              'px_accuracy': px_accuracy,
              'precision': precision,
              'recall': recall,
              'iou_score': iou_score,
              'fmeasure':fmeasure
             }
    return report


def log_batch_statistics(batch_number,batch_labels,batch_preds,phase,loss,since,num_batches,period=500):
    if batch_number % period == 0:
        iou_score = segmentation_report(batch_preds,batch_labels)
        time_elapsed = time.time() - since

        if phase == 'train':
            telegram_bot_sendtext('TRAINING BATCH')
        else:
            telegram_bot_sendtext('VALIDATION BATCH')
            
        telegram_bot_sendtext('-'*50)
        telegram_bot_sendtext(f'\n{batch_number}/{num_batches-1}:')
        telegram_bot_sendtext(f'Total Time Elapsed: {time_elapsed/60:.2f} mins')
        telegram_bot_sendtext(f'Batch Loss: {loss.item():.4f}\n')
        telegram_bot_sendtext(f"``` IoU_score:{iou_score}\n ```")
        telegram_bot_sendtext('-'*50)

def break_time_limit(start_time,time_limit=28080):
    time_elapsed = time()-start_time
    if time_elapsed > time_limit:
        sys.exit()

# Model Setup

In [ ]:
#learning policy params
grayscale = True
if grayscale == True:
    in_channels = 2
else: in_channels = 6

N_EPOCHS = 25

# turning on GPU if possible
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# cleaning GPU
gc.collect() 
torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = True

# mean percentage of positives is 6.5% from the frame, median is 3.4%, so weights for bce loss are required.
weights = torch.Tensor([28]).to(device) 

#criterion = torch.nn.BCEWithLogitsLoss(pos_weight = weights) 
criterion = IoULoss()


### Change detection FC model

I decided to relate on a following articale: Daudt, R.C., Le Saux, B. and Boulch, A., 2018, October. Fully convolutional siamese networks for change detection. In 2018 25th IEEE International Conference on Image Processing (ICIP) (pp. 4063-4067). IEEE.https://rcdaudt.github.io/files/2018icip-fully-convolutional.pdf .  
It desctibes neural networks architectures for CD specifically, which authors consider as most effective in cases of training from zero, without transfer learning or fine tuning of pretrained models.  
These are:
* FC-EF (Fully Convolutional - Early Fusion)
* FC_Siam_conc
* FC_Siam_diff  


The first one involves feeding two of the concatenated rasters and their joint processing, the second and third involves separate treatment of the raster inputs at the beginning, during the convolutional stage, with identical branches with the shared weights and bias. Both outputs of individual branches are concatenated and fed to the input of a single network, ending with a layer of softmax or sigmoid transformation. Siam_conc and Siam_diff differ from each other in the mechanism of implementation of the feature skipping, in the first case it is more intuitive - intermediate features from the two branches of the conv stage are concatenated with the output at the respective upconv stages. In the case of Siam_diff, instead of two sets, the absolute difference between the intermediate features is concatenated. In the conclusion to the article it is proposed to use FC_Siam_diff as the most productive approach (although it is noted that all the above-mentioned methods are superior to the use of, for example, Transfer Learning) the next most productive one is Early Fusion.

In [ ]:
import torch.nn.functional as F

class Unet(nn.Module):
    """EF segmentation network."""

    def __init__(self, input_nbr, label_nbr):
        super(Unet, self).__init__()

        self.input_nbr = input_nbr

        self.conv11 = nn.Conv2d(input_nbr, 16, kernel_size=3, padding=1)
        self.bn11 = nn.BatchNorm2d(16)
        self.do11 = nn.Dropout2d(p=0.2)
        self.conv12 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.bn12 = nn.BatchNorm2d(16)
        self.do12 = nn.Dropout2d(p=0.2)

        self.conv21 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn21 = nn.BatchNorm2d(32)
        self.do21 = nn.Dropout2d(p=0.2)
        self.conv22 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn22 = nn.BatchNorm2d(32)
        self.do22 = nn.Dropout2d(p=0.2)

        self.conv31 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn31 = nn.BatchNorm2d(64)
        self.do31 = nn.Dropout2d(p=0.2)
        self.conv32 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn32 = nn.BatchNorm2d(64)
        self.do32 = nn.Dropout2d(p=0.2)
        self.conv33 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn33 = nn.BatchNorm2d(64)
        self.do33 = nn.Dropout2d(p=0.2)

        self.conv41 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn41 = nn.BatchNorm2d(128)
        self.do41 = nn.Dropout2d(p=0.2)
        self.conv42 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn42 = nn.BatchNorm2d(128)
        self.do42 = nn.Dropout2d(p=0.2)
        self.conv43 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn43 = nn.BatchNorm2d(128)
        self.do43 = nn.Dropout2d(p=0.2)


        self.upconv4 = nn.ConvTranspose2d(128, 128, kernel_size=3, padding=1, stride=2, output_padding=1)

        self.conv43d = nn.ConvTranspose2d(256, 128, kernel_size=3, padding=1)
        self.bn43d = nn.BatchNorm2d(128)
        self.do43d = nn.Dropout2d(p=0.2)
        self.conv42d = nn.ConvTranspose2d(128, 128, kernel_size=3, padding=1)
        self.bn42d = nn.BatchNorm2d(128)
        self.do42d = nn.Dropout2d(p=0.2)
        self.conv41d = nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1)
        self.bn41d = nn.BatchNorm2d(64)
        self.do41d = nn.Dropout2d(p=0.2)

        self.upconv3 = nn.ConvTranspose2d(64, 64, kernel_size=3, padding=1, stride=2, output_padding=1)

        self.conv33d = nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1)
        self.bn33d = nn.BatchNorm2d(64)
        self.do33d = nn.Dropout2d(p=0.2)
        self.conv32d = nn.ConvTranspose2d(64, 64, kernel_size=3, padding=1)
        self.bn32d = nn.BatchNorm2d(64)
        self.do32d = nn.Dropout2d(p=0.2)
        self.conv31d = nn.ConvTranspose2d(64, 32, kernel_size=3, padding=1)
        self.bn31d = nn.BatchNorm2d(32)
        self.do31d = nn.Dropout2d(p=0.2)

        self.upconv2 = nn.ConvTranspose2d(32, 32, kernel_size=3, padding=1, stride=2, output_padding=1)

        self.conv22d = nn.ConvTranspose2d(64, 32, kernel_size=3, padding=1)
        self.bn22d = nn.BatchNorm2d(32)
        self.do22d = nn.Dropout2d(p=0.2)
        self.conv21d = nn.ConvTranspose2d(32, 16, kernel_size=3, padding=1)
        self.bn21d = nn.BatchNorm2d(16)
        self.do21d = nn.Dropout2d(p=0.2)

        self.upconv1 = nn.ConvTranspose2d(16, 16, kernel_size=3, padding=1, stride=2, output_padding=1)

        self.conv12d = nn.ConvTranspose2d(32, 16, kernel_size=3, padding=1)
        self.bn12d = nn.BatchNorm2d(16)
        self.do12d = nn.Dropout2d(p=0.2)
        self.conv11d = nn.ConvTranspose2d(16, label_nbr, kernel_size=3, padding=1)

        self.sm = nn.Sigmoid()

    def forward(self, x1, x2):

        x = torch.cat((x1, x2), 1)

        """Forward method."""
        # Stage 1
        x11 = self.do11(F.relu(self.bn11(self.conv11(x))))
        x12 = self.do12(F.relu(self.bn12(self.conv12(x11))))
        x1p = F.max_pool2d(x12, kernel_size=2, stride=2)

        # Stage 2
        x21 = self.do21(F.relu(self.bn21(self.conv21(x1p))))
        x22 = self.do22(F.relu(self.bn22(self.conv22(x21))))
        x2p = F.max_pool2d(x22, kernel_size=2, stride=2)

        # Stage 3
        x31 = self.do31(F.relu(self.bn31(self.conv31(x2p))))
        x32 = self.do32(F.relu(self.bn32(self.conv32(x31))))
        x33 = self.do33(F.relu(self.bn33(self.conv33(x32))))
        x3p = F.max_pool2d(x33, kernel_size=2, stride=2)

        # Stage 4
        x41 = self.do41(F.relu(self.bn41(self.conv41(x3p))))
        x42 = self.do42(F.relu(self.bn42(self.conv42(x41))))
        x43 = self.do43(F.relu(self.bn43(self.conv43(x42))))
        x4p = F.max_pool2d(x43, kernel_size=2, stride=2)


        # Stage 4d
        x4d = self.upconv4(x4p)
        pad4 = ReplicationPad2d((0, x43.size(3) - x4d.size(3), 0, x43.size(2) - x4d.size(2)))
        x4d = torch.cat((pad4(x4d), x43), 1)
        x43d = self.do43d(F.relu(self.bn43d(self.conv43d(x4d))))
        x42d = self.do42d(F.relu(self.bn42d(self.conv42d(x43d))))
        x41d = self.do41d(F.relu(self.bn41d(self.conv41d(x42d))))

        # Stage 3d
        x3d = self.upconv3(x41d)
        pad3 = ReplicationPad2d((0, x33.size(3) - x3d.size(3), 0, x33.size(2) - x3d.size(2)))
        x3d = torch.cat((pad3(x3d), x33), 1)
        x33d = self.do33d(F.relu(self.bn33d(self.conv33d(x3d))))
        x32d = self.do32d(F.relu(self.bn32d(self.conv32d(x33d))))
        x31d = self.do31d(F.relu(self.bn31d(self.conv31d(x32d))))

        # Stage 2d
        x2d = self.upconv2(x31d)
        pad2 = ReplicationPad2d((0, x22.size(3) - x2d.size(3), 0, x22.size(2) - x2d.size(2)))
        x2d = torch.cat((pad2(x2d), x22), 1)
        x22d = self.do22d(F.relu(self.bn22d(self.conv22d(x2d))))
        x21d = self.do21d(F.relu(self.bn21d(self.conv21d(x22d))))

        # Stage 1d
        x1d = self.upconv1(x21d)
        pad1 = ReplicationPad2d((0, x12.size(3) - x1d.size(3), 0, x12.size(2) - x1d.size(2)))
        x1d = torch.cat((pad1(x1d), x12), 1)
        x12d = self.do12d(F.relu(self.bn12d(self.conv12d(x1d))))
        x11d = self.conv11d(x12d)

        return self.sm(x11d)

class SiamUnet_diff(nn.Module):
    """SiamUnet_diff segmentation network."""

    def __init__(self, input_nbr, label_nbr):
        super(SiamUnet_diff, self).__init__()

        self.input_nbr = input_nbr

        self.conv11 = nn.Conv2d(input_nbr, 16, kernel_size=3, padding=1)
        self.bn11 = nn.BatchNorm2d(16)
        self.do11 = nn.Dropout2d(p=0.2)
        self.conv12 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.bn12 = nn.BatchNorm2d(16)
        self.do12 = nn.Dropout2d(p=0.2)

        self.conv21 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn21 = nn.BatchNorm2d(32)
        self.do21 = nn.Dropout2d(p=0.2)
        self.conv22 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn22 = nn.BatchNorm2d(32)
        self.do22 = nn.Dropout2d(p=0.2)

        self.conv31 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn31 = nn.BatchNorm2d(64)
        self.do31 = nn.Dropout2d(p=0.2)
        self.conv32 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn32 = nn.BatchNorm2d(64)
        self.do32 = nn.Dropout2d(p=0.2)
        self.conv33 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn33 = nn.BatchNorm2d(64)
        self.do33 = nn.Dropout2d(p=0.2)

        self.conv41 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn41 = nn.BatchNorm2d(128)
        self.do41 = nn.Dropout2d(p=0.2)
        self.conv42 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn42 = nn.BatchNorm2d(128)
        self.do42 = nn.Dropout2d(p=0.2)
        self.conv43 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn43 = nn.BatchNorm2d(128)
        self.do43 = nn.Dropout2d(p=0.2)

        self.upconv4 = nn.ConvTranspose2d(128, 128, kernel_size=3, padding=1, stride=2, output_padding=1)

        self.conv43d = nn.ConvTranspose2d(256, 128, kernel_size=3, padding=1)
        self.bn43d = nn.BatchNorm2d(128)
        self.do43d = nn.Dropout2d(p=0.2)
        self.conv42d = nn.ConvTranspose2d(128, 128, kernel_size=3, padding=1)
        self.bn42d = nn.BatchNorm2d(128)
        self.do42d = nn.Dropout2d(p=0.2)
        self.conv41d = nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1)
        self.bn41d = nn.BatchNorm2d(64)
        self.do41d = nn.Dropout2d(p=0.2)

        self.upconv3 = nn.ConvTranspose2d(64, 64, kernel_size=3, padding=1, stride=2, output_padding=1)

        self.conv33d = nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1)
        self.bn33d = nn.BatchNorm2d(64)
        self.do33d = nn.Dropout2d(p=0.2)
        self.conv32d = nn.ConvTranspose2d(64, 64, kernel_size=3, padding=1)
        self.bn32d = nn.BatchNorm2d(64)
        self.do32d = nn.Dropout2d(p=0.2)
        self.conv31d = nn.ConvTranspose2d(64, 32, kernel_size=3, padding=1)
        self.bn31d = nn.BatchNorm2d(32)
        self.do31d = nn.Dropout2d(p=0.2)

        self.upconv2 = nn.ConvTranspose2d(32, 32, kernel_size=3, padding=1, stride=2, output_padding=1)

        self.conv22d = nn.ConvTranspose2d(64, 32, kernel_size=3, padding=1)
        self.bn22d = nn.BatchNorm2d(32)
        self.do22d = nn.Dropout2d(p=0.2)
        self.conv21d = nn.ConvTranspose2d(32, 16, kernel_size=3, padding=1)
        self.bn21d = nn.BatchNorm2d(16)
        self.do21d = nn.Dropout2d(p=0.2)

        self.upconv1 = nn.ConvTranspose2d(16, 16, kernel_size=3, padding=1, stride=2, output_padding=1)

        self.conv12d = nn.ConvTranspose2d(32, 16, kernel_size=3, padding=1)
        self.bn12d = nn.BatchNorm2d(16)
        self.do12d = nn.Dropout2d(p=0.2)
        self.conv11d = nn.ConvTranspose2d(16, label_nbr, kernel_size=3, padding=1)

        self.sm = nn.Sigmoid()

    def forward(self, x1, x2):


        """Forward method."""
        # Stage 1
        x11 = self.do11(F.relu(self.bn11(self.conv11(x1))))
        x12_1 = self.do12(F.relu(self.bn12(self.conv12(x11))))
        x1p = F.max_pool2d(x12_1, kernel_size=2, stride=2)


        # Stage 2
        x21 = self.do21(F.relu(self.bn21(self.conv21(x1p))))
        x22_1 = self.do22(F.relu(self.bn22(self.conv22(x21))))
        x2p = F.max_pool2d(x22_1, kernel_size=2, stride=2)

        # Stage 3
        x31 = self.do31(F.relu(self.bn31(self.conv31(x2p))))
        x32 = self.do32(F.relu(self.bn32(self.conv32(x31))))
        x33_1 = self.do33(F.relu(self.bn33(self.conv33(x32))))
        x3p = F.max_pool2d(x33_1, kernel_size=2, stride=2)

        # Stage 4
        x41 = self.do41(F.relu(self.bn41(self.conv41(x3p))))
        x42 = self.do42(F.relu(self.bn42(self.conv42(x41))))
        x43_1 = self.do43(F.relu(self.bn43(self.conv43(x42))))
        x4p = F.max_pool2d(x43_1, kernel_size=2, stride=2)

        ####################################################
        # Stage 1
        x11 = self.do11(F.relu(self.bn11(self.conv11(x2))))
        x12_2 = self.do12(F.relu(self.bn12(self.conv12(x11))))
        x1p = F.max_pool2d(x12_2, kernel_size=2, stride=2)


        # Stage 2
        x21 = self.do21(F.relu(self.bn21(self.conv21(x1p))))
        x22_2 = self.do22(F.relu(self.bn22(self.conv22(x21))))
        x2p = F.max_pool2d(x22_2, kernel_size=2, stride=2)

        # Stage 3
        x31 = self.do31(F.relu(self.bn31(self.conv31(x2p))))
        x32 = self.do32(F.relu(self.bn32(self.conv32(x31))))
        x33_2 = self.do33(F.relu(self.bn33(self.conv33(x32))))
        x3p = F.max_pool2d(x33_2, kernel_size=2, stride=2)

        # Stage 4
        x41 = self.do41(F.relu(self.bn41(self.conv41(x3p))))
        x42 = self.do42(F.relu(self.bn42(self.conv42(x41))))
        x43_2 = self.do43(F.relu(self.bn43(self.conv43(x42))))
        x4p = F.max_pool2d(x43_2, kernel_size=2, stride=2)



        # Stage 4d
        x4d = self.upconv4(x4p)
        pad4 = ReplicationPad2d((0, x43_1.size(3) - x4d.size(3), 0, x43_1.size(2) - x4d.size(2)))
        x4d = torch.cat((pad4(x4d), torch.abs(x43_1 - x43_2)), 1)
        x43d = self.do43d(F.relu(self.bn43d(self.conv43d(x4d))))
        x42d = self.do42d(F.relu(self.bn42d(self.conv42d(x43d))))
        x41d = self.do41d(F.relu(self.bn41d(self.conv41d(x42d))))

        # Stage 3d
        x3d = self.upconv3(x41d)
        pad3 = ReplicationPad2d((0, x33_1.size(3) - x3d.size(3), 0, x33_1.size(2) - x3d.size(2)))
        x3d = torch.cat((pad3(x3d), torch.abs(x33_1 - x33_2)), 1)
        x33d = self.do33d(F.relu(self.bn33d(self.conv33d(x3d))))
        x32d = self.do32d(F.relu(self.bn32d(self.conv32d(x33d))))
        x31d = self.do31d(F.relu(self.bn31d(self.conv31d(x32d))))

        # Stage 2d
        x2d = self.upconv2(x31d)
        pad2 = ReplicationPad2d((0, x22_1.size(3) - x2d.size(3), 0, x22_1.size(2) - x2d.size(2)))
        x2d = torch.cat((pad2(x2d), torch.abs(x22_1 - x22_2)), 1)
        x22d = self.do22d(F.relu(self.bn22d(self.conv22d(x2d))))
        x21d = self.do21d(F.relu(self.bn21d(self.conv21d(x22d))))

        # Stage 1d
        x1d = self.upconv1(x21d)
        pad1 = ReplicationPad2d((0, x12_1.size(3) - x1d.size(3), 0, x12_1.size(2) - x1d.size(2)))
        x1d = torch.cat((pad1(x1d), torch.abs(x12_1 - x12_2)), 1)
        x12d = self.do12d(F.relu(self.bn12d(self.conv12d(x1d))))
        x11d = self.conv11d(x12d)

        return self.sm(x11d)


In [ ]:
!pip install segmentation_models_pytorch
import segmentation_models_pytorch as smp

In [ ]:
if grayscale == True:
    net, net_name = smp.Unet('resnet34', in_channels = 2, activation='sigmoid'), 'FC-EF'
    #net, net_name = SiamUnet_diff(1,1), 'FC-Siam-diff'
    #net, net_name = SiamUnet_conc(1,1), 'FC-Siam-conc'
    #net, net_name = Unet(2,1), 'FC-EF'
    
else:
    net, net_name = smp.Unet('resnet34', in_channels = 6, activation='sigmoid'), 'FC-EF'
    #net, net_name = SiamUnet_diff(3, 1), 'FC-Siam-diff'
    #net, net_name = SiamUnet_conc(3,1), 'FC-Siam-conc'
    #net, net_name = Unet(3*2,1), 'FC-EF'
net = net.to(device)

In [ ]:
#defining datasets, samplers and dataloaders
datasets = {x:TorchDataset(root_folder = root_folder,df = df_dict[x],aug = None, preproc = None, grayscale = grayscale) for x in ['train','test','valid']}

samplers = {'train':BalancedSampler(datasets['train'], percentage = 1), 
            'test':BalancedSampler(datasets['test'], percentage = 1),
            'valid':BalancedSampler(datasets['valid'], percentage = 1),
            'sanity':BalancedSampler(datasets['train'], percentage = 1)}

dataloaders = {x: DataLoader(dataset=datasets[x],
                             batch_size=BATCH_SIZE,
                             sampler=samplers[x],
                             num_workers=16) for x in ['train','test','valid']}

dataset_sizes = {x: len(datasets[x]) for x in ['train', 'test', 'valid']}



# Train function

In [ ]:
# net = torch.load('../input/satellite-models/netFC-Siam-diff-best_epoch-1_iou_score-0.023448613236376363.tar')

In [ ]:
def train(net, n_epochs = 25):
    print('epoch,train_loss,train_iou,test_loss,test_iou',file=open('loss_log.txt', 'a'))
    start_time = time()
    # telegram_bot_sendtext(f'Training started')
    scaler = GradScaler()
        
    iou = -1
    best_iou = -1
    
    lss = 100000000
    best_lss = 100000000
    
    train_epoch_iou = 0
    test_epoch_iou = 0
    
    
    #defining optimizer and scheduler
    optimizer = torch.optim.Adam(net.parameters(), lr = 0.005)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.95)
    

    for epoch_index in tqdm(range(n_epochs)):
        print('Epoch: ' + str(epoch_index + 1) + ' of ' + str(n_epochs))
        train_running_loss = 0.0
        train_running_iou = 0.0
        test_running_loss = 0.0
        test_running_iou = 0.0
        
        for phase in ['train','test']:
            if phase == 'train':
                net.train()  # Set model to training mode
            else:
                net.eval()   # Set model to evaluate mode

            num_batches = len(dataloaders[phase])
            for batch_index, batch in enumerate(tqdm(dataloaders[phase])):
                torch.cuda.empty_cache()
                I1 = Variable(batch['I1'].float().to(device))
                I2 = Variable(batch['I2'].float().to(device))
                labels = Variable(batch['label'].float().to(device))

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    with autocast():
                        # outputs = net(I1, I2)
                        outputs = net(torch.stack((I1, I2), axis = 1).squeeze()) # fusion for smp.Unet
                        loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs.data, 1)
                    del(_,outputs)
                    if phase == 'train':
                        scaler.scale(loss).backward()
                        scaler.step(optimizer)
                        optimizer.step()
                        scaler.update()
                    del(I1, I2)
#                     except Exception as e:
#                             msg = log_traceback(e)
#                             telegram_bot_sendtext(f'failed at batch {batch_index}, with message: {msg}')
#                             raise e
#                             break
#                     log_batch_statistics(batch_index,labels.data.long(),outputs.data.long(),phase,loss=loss,num_batches=num_batches,since=start_time)
                if phase == 'train':
                    train_running_loss += loss.item()
                    # train_running_iou += iou_pytorch(preds.int().to('cpu'), labels.int().to('cpu')).item()
                else:
                    test_running_loss += loss.item()
                    # test_running_iou += iou_pytorch(preds.int().to('cpu'), labels.int().to('cpu')).item()
                                
            if phase == 'train':
                scheduler.step()
                train_epoch_loss = train_running_loss / dataset_sizes[phase] * BATCH_SIZE
                # train_epoch_iou = train_running_iou / dataset_sizes[phase] * BATCH_SIZE
            else:
                test_epoch_loss = test_running_loss / dataset_sizes[phase] * BATCH_SIZE
                # test_epoch_iou = test_running_iou / dataset_sizes[phase] * BATCH_SIZE
            print(f'{epoch_index},{train_epoch_loss}, {train_epoch_iou},{test_epoch_loss},{test_epoch_iou}', file=open('loss_log.txt', 'a'))
#             print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=20), file=open("profiler.txt", "a"))
#             telegram_send_file('./profiler.txt')
            
#             if epoch_iou > best_iou:
#                 best_iou = epoch_iou
#                 save_str = f'./net{net_name}-best_epoch-' + str(epoch_index + 1) + '_iou_score-' + str(best_iou) + '.pth.tar'
#                 torch.save(net, save_str)
#                 telegram_send_file(save_str)
#                 print('Model saved!')
        
            if (phase  == 'train') and (train_epoch_loss < best_lss):
                best_lss = train_epoch_loss
                save_str = f'./net{net_name}-best_epoch-' + str(epoch_index + 1) + '_loss-' + str(best_lss) + '.pth.tar'
                torch.save(net, save_str)
                telegram_send_file(save_str)
                print('Model saved!')
            
            if (phase == 'test') and (test_epoch_loss < best_lss):
                best_lss = test_epoch_loss
                save_str = f'./net{net_name}-best_epoch-' + str(epoch_index + 1) + '_loss-' + str(best_lss) + '.pth.tar'
                torch.save(net, save_str)
                telegram_send_file(save_str)
                print('Model saved!')
        
    
    model_str = './model.pth.tar'
    torch.save(net, model_str)
    telegram_send_file(model_str)
    print('Model saved!')
    time_elapsed = time() - start_time
    print()
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val change_accuracy: {best_fm:4f}')
    
    return net
        

In [ ]:
model = train(net,2)  

In [ ]:
dataset_sizes['train']

In [ ]:
dataset_sizes['valid']

# Results visualization

In [ ]:
model_path = '../input/satellite-models/netFC-Siam-diff-best_epoch-1_iou_score-0.825063088258398.pth.tar'
model = torch.load(model_path)

In [ ]:
model = model.cuda()

In [ ]:
model.eval()

In [ ]:
def val_model(model):
    iou_list = []
    for batch_index, batch in enumerate(tqdm(dataloaders['valid'])):
        I1 = Variable(batch['I1'].float().to(device))
        I2 = Variable(batch['I2'].float().to(device))
        labels = Variable(batch['label'].int().to(device))
        outputs = model(I1,I2)
        iou_list.append(iou_pytorch(outputs.int().to('cpu'), labels.to('cpu')))
    print(np.mean(iou_list)) 

In [ ]:
val_model(model)

In [ ]:
batch = next(iter(dataloaders['valid']))
n = np.random.randint(BATCH_SIZE)
_, preds = torch.max(model(batch['I1'].cuda(),batch['I2'].cuda()).data,1)

In [ ]:
fig, axs = plt.subplots(1,4)
fig.suptitle('Random chip prediction')
fig.tight_layout() 
axs[0].imshow(batch['I1'][n].squeeze())
axs[1].imshow(batch['I2'][n].squeeze())
axs[2].imshow(batch['label'][n].squeeze())
axs[3].imshow(preds[n].cpu())


# Sandbox for debuging

In [ ]:
torch.stack((datasets['train'][0]['I1'],datasets['train'][0]['I1']),axis = 1).squeeze().shape